# Dependencies

In [ ]:
import serial
import pickle
from pathlib import Path
import numpy as np
import regex as re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

columns=["0deg","18deg","36deg","54deg","72deg","90deg"]

def columns_formated():
    res = []
    for col in columns:
        res.append(re.sub("deg", "°", col))
    return res

In [ ]:
robotOut = []

with serial.Serial('/dev/ttyACM0', 9600) as ser:
    while len(robotOut) < 300:
        line = ser.readline().decode("utf-8")
        print(line, end="")
        line = re.sub("\S+:", "", line)
        line = re.sub("\r\n", "", line)
        robotOut.append(float(line))
    
    

In [ ]:
print(len(robotOut))
plt.plot(robotOut)

In [ ]:
with open("data/90deg/l_90deg_run1.pkl", "wb") as file:
            pickle.dump(robotOut, file)

In [ ]:
with open("data/36deg/36deg_run1.pkl", "rb") as file:
            robotOut = pickle.load(file)

In [ ]:
fig, axs = plt.subplots(figsize=(4*3,3*2), nrows=2, ncols = 3)

def getResults():

    results = np.empty((6,5,300))

    def setResults(results, robotOut, index, file):
        run = re.sub("\S+n", "", file.parts[2])
        run = re.sub("\.\S+", "", run)
        results[index, int(run)-1] = robotOut

    for file in Path().rglob("*.pkl"):
        with open(file, "rb") as f:
            robotOut = pickle.load(f)
            
        if file.parts[1] in columns:
            setResults(results, robotOut, columns.index(file.parts[1]), file)
            
    return results
        
results = getResults()     
results = np.average(results, axis=(1)) 


for i, val in enumerate(columns):
    col = i//3
    row = i - (col*3)
    
    axs[col,row].plot(results[i], label=re.sub("deg", "°", val))


for axi in axs:
    for ax in axi:
        ax.set_ylim(-0.4,0.7)
        ax.legend()

fig.text(0.5, 0.06, 'Recorder Index', ha='center', va='center', fontsize='large')
fig.text(0.075, 0.5, 'Average Error', ha='center', va='center', rotation='vertical', fontsize='large')
fig.savefig("feedback_error.svg")

In [ ]:
results = getResults()
results = np.average(results, axis=(1))


fig, ax = plt.subplots(figsize=(8,3))

results = pd.DataFrame(results.T, columns=columns_formated())
# print(results.shape)

# axs[0,0].boxplot(results[0])
sns.boxplot(ax=ax, data=results, orient="h")
ax.set_ylabel("Degree")
ax.set_xlabel("Average Error")
fig.savefig("feedback_error_boxplot.svg")
# axs[0,0] = ax
# axs[0,0].boxplot(results[0])

In [ ]:
print(len(robotOut))
robotOut

In [ ]:
pwm_20 = [-9.1, -12.4, -8.6, -10.3, -15.4, -11.2, 10.0, -6.0, -8.1, 1.1]  
pwm_40 = [-12.2, 10.1, 8.1, -8.2, 0.2, 8.5, -6.4, 6.2, 1.1, 0.8]


pwm_20_dataframe = pd.DataFrame({'20':pwm_20})
pwm_40_dataframe = pd.DataFrame({'40':pwm_40})


combined = pd.concat([ pwm_20_dataframe, pwm_40_dataframe], axis=0)


plot = sns.boxplot(data=combined)
plot.set(xlabel ="PWM Value", ylabel = "Travel Error (mm)", title ='Fictional Results')
plt.show()